In [4]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from tabulate import tabulate
from collections import Counter
import pandas as pd

In [5]:
dataset = pd.read_csv("customer_complaints_1.csv")

In [30]:

dataset = dataset['text']str.lower()

TypeError: list indices must be integers or slices, not str

In [7]:
tokenized_dataset = [doc.split() for doc in dataset]

In [8]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amaknabil2001\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amaknabil2001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
stopwords = nltk.corpus.stopwords.words('english')

In [10]:
def remove_stopwords(text):
    output = []
    for i in text:
        if i not in stopwords:
            output.append(i)
    return output
# Assuming you have defined the remove_stopwords function and tokenized_dataset is a list containing tokenized text

# Apply remove_stopwords function to each element in tokenized_dataset using list comprehension
tokenized_dataset = [remove_stopwords(tokens) for tokens in tokenized_dataset]



In [11]:

#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer

In [12]:
#defining the object for stemming
porter_stemmer = PorterStemmer()

In [13]:
#defining a function for stemming
def stemming(text):
    stem_text = []
    for word in text:
        stemmed_word = porter_stemmer.stem(word)
        stem_text.append(stemmed_word)
    return stem_text

In [14]:
tokenized_dataset = [stemming(tokens) for tokens in tokenized_dataset]

In [15]:
print(tokenized_dataset)

[['author'], ['posted_on'], ['rate'], ['text']]


In [16]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amaknabil2001\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
#importing the Lemmatizer function from nltk library
from nltk.stem import WordNetLemmatizer

In [18]:

#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

In [19]:
def lemmatizer(text):
    lemm_text = []
    for word in text:
        lemmatized_word = wordnet_lemmatizer.lemmatize(word)
        lemm_text.append(lemmatized_word)
    return lemm_text

In [20]:
tokenized_dataset = [lemmatizer(tokens) for tokens in tokenized_dataset]

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Flatten the list of lists into a single list of strings
flattened_tokenized_dataset = [" ".join(tokens) for tokens in tokenized_dataset]

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the tokenized dataset
X = vectorizer.fit_transform(flattened_tokenized_dataset)


In [22]:
k = 2 # Define the number of clusters
km = KMeans(n_clusters=k)
km.fit(X)

C:\Users\amaknabil2001\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=2)

In [23]:
# Predict the clusters for each document
y_pred = km.predict(X)

# Display the document and its predicted cluster in a table
table_data = [["Document", "Predicted Cluster"]]
table_data.extend([[doc, cluster] for doc, cluster in zip(dataset, y_pred)])
print(tabulate(table_data, headers="firstrow"))

# Page 3 CISB5123 Nur Laila Ab Ghani

# Print top terms per cluster
print("\nTop terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
    print()


Document      Predicted Cluster
----------  -------------------
author                        0
posted_on                     0
rating                        1
text                          0

Top terms per cluster:
Cluster 0:
 text
 posted_on
 author
 rate

Cluster 1:
 rate
 text
 posted_on
 author



In [24]:
# Calculate purity
total_samples = len(y_pred)
cluster_label_counts = [Counter(y_pred)]
purity = sum(max(cluster.values()) for cluster in cluster_label_counts) / total_samples
print("Purity:", purity)

Purity: 0.75


In [25]:
import numpy as np
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from tabulate import tabulate
from collections import Counter

In [26]:
word2vec_model = Word2Vec(sentences=tokenized_dataset, vector_size=100, 
window=5, min_count=1, workers=4)

In [28]:
import numpy as np

# Initialize an empty list to store the word embeddings
embeddings = []

# Iterate over each document in the dataset
for doc in dataset:
    # Initialize an empty list to store word embeddings for words in the document
    doc_embeddings = []
    # Iterate over each word in the document
    for word in doc.split():
        # Check if the word is in the vocabulary of the word2vec model
        if word in word2vec_model.wv:
            # If the word is in the vocabulary, append its word embedding to the list
            doc_embeddings.append(word2vec_model.wv[word])
    # Calculate the mean of word embeddings for the document and append it to the embeddings list
    if doc_embeddings:
        embeddings.append(np.mean(doc_embeddings, axis=0))
    else:
        # If no word embeddings were found for the document, append a zero vector
        embeddings.append(np.zeros(word2vec_model.vector_size))

# Convert the list of embeddings into a numpy array
X = np.array(embeddings)


In [29]:
k = 2 # Define the number of clusters
km = KMeans(n_clusters=k)
km.fit(X)
# Predict the clusters for each document
y_pred = km.predict(X)
# Tabulate the document and predicted cluster
table_data = [["Document", "Predicted Cluster"]]
table_data.extend([[doc, cluster] for doc, cluster in zip(dataset, y_pred)])
print(tabulate(table_data, headers="firstrow"))

# Calculate purity
total_samples = len(y_pred)
cluster_label_counts = [Counter(y_pred)]
purity = sum(max(cluster.values()) for cluster in cluster_label_counts) / total_samples
print("Purity:", purity)

C:\Users\amaknabil2001\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\amaknabil2001\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Document      Predicted Cluster
----------  -------------------
author                        0
posted_on                     1
rating                        1
text                          1
Purity: 0.75
